In [1]:
import time
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch
if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True

In [2]:
from torch.cuda import random
device = torch.device("cuda0" if torch.cuda.is_available() else "cpu")

random_seed = 1
learning_rate = 0.1
num_epochs = 10
batch_size = 64
dropout_prob = 0.5

num_features = 784
num_hidden_1 = 128
num_hidden_2 = 256
num_classes = 10

train_dataset = datasets.MNIST(root="data",train=True,transform=transforms.ToTensor(),download=True)
test_dataset = datasets.MNIST(root="data",train=False,transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

for images, labels in train_loader:
  print("Image batch dimensions:",images.shape)
  print("Image label dimensions:",labels.shape)
  break

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 495kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.66MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.71MB/s]


Image batch dimensions: torch.Size([64, 1, 28, 28])
Image label dimensions: torch.Size([64])


In [3]:
class MLP(torch.nn.Module):
  def __init__(self,num_features,num_classes):
    super(MLP,self).__init__()
    self.linear_1 = torch.nn.Linear(num_features,num_hidden_1)
    self.linear_1.weight.detach().normal_(0.0,0.1)
    self.linear_1.bias.detach().zero_()

    self.linear_2 = torch.nn.Linear(num_hidden_1,num_hidden_2)
    self.linear_2.weight.detach().normal_(0.0,0.1)
    self.linear_2.bias.detach().zero_()

    self.linear_out = torch.nn.Linear(num_hidden_2,num_classes)
    self.linear_out.weight.detach().normal_(0.0,0.1)
    self.linear_out.bias.detach().zero_()

  def forward(self,x):
    out = self.linear_1(x)
    out = F.relu(out)
    out = F.dropout(out, p=dropout_prob, training=self.training)
    out = self.linear_2(out)
    out = F.relu(out)
    out = F.dropout(out, p=dropout_prob, training=self.training)
    logits = self.linear_out(out)
    probas = F.softmax(logits,dim=1)
    return logits,probas

torch.manual_seed(random_seed)
model = MLP(num_features=num_features, num_classes=num_classes)
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
def compute_accuracy(net,data_loader):
  net.eval()
  correct_pred, num_examples = 0, 0
  with torch.no_grad():
    for features, targets in data_loader:
      features = features.view(-1,28*28).to(device)
      targets = targets.to(device)
      logits, probas = net(features)
      _,predicted_labels = torch.max(probas,1)
      num_examples += targets.size(0)
      correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100
start_time = time.time()
for epoch in range(num_epochs):
  model.train()
  for batch_idx, (features, targets) in enumerate(train_loader):
    features = features.view(-1,28*28).to(device)
    targets = targets.to(device)

    logits, probas = model(features)
    cost = F.cross_entropy(logits,targets)
    optimizer.zero_grad()

    cost.backward()

    optimizer.step()

    if not batch_idx % 50:
      print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f'
                   %(epoch+1, num_epochs, batch_idx,
                     len(train_loader), cost))
  print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, num_epochs,
          compute_accuracy(model, train_loader)))

  print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 000/938 | Cost: 0.5800
Epoch: 001/010 | Batch 050/938 | Cost: 0.4650
Epoch: 001/010 | Batch 100/938 | Cost: 0.4873
Epoch: 001/010 | Batch 150/938 | Cost: 0.5862
Epoch: 001/010 | Batch 200/938 | Cost: 0.2436
Epoch: 001/010 | Batch 250/938 | Cost: 0.2921
Epoch: 001/010 | Batch 300/938 | Cost: 0.3975
Epoch: 001/010 | Batch 350/938 | Cost: 0.4826
Epoch: 001/010 | Batch 400/938 | Cost: 0.3383
Epoch: 001/010 | Batch 450/938 | Cost: 0.4192
Epoch: 001/010 | Batch 500/938 | Cost: 0.5176
Epoch: 001/010 | Batch 550/938 | Cost: 0.2289
Epoch: 001/010 | Batch 600/938 | Cost: 0.3065
Epoch: 001/010 | Batch 650/938 | Cost: 0.2995
Epoch: 001/010 | Batch 700/938 | Cost: 0.4396
Epoch: 001/010 | Batch 750/938 | Cost: 0.2114
Epoch: 001/010 | Batch 800/938 | Cost: 0.4689
Epoch: 001/010 | Batch 850/938 | Cost: 0.1430
Epoch: 001/010 | Batch 900/938 | Cost: 0.4208
Epoch: 001/010 training accuracy: 94.62%
Time elapsed: 0.30 min
Epoch: 002/010 | Batch 000/938 | Cost: 0.5103
Epoch: 002/010 |

In [6]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 97.09%
